In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [11]:
from quantum_tools.examples import symbolic_contexts
from quantum_tools.symmetries.workspace import get_contraction_elements
from quantum_tools.examples.prob_dists import *
from quantum_tools.utilities.job_queuer_async import JobContext
from quantum_tools.config import *
from quantum_tools.visualization.hypergraph import *
from quantum_tools.visualization.transversal_inequalities import *
from quantum_tools.hypergraph.hypergraph_transversals import *
from quantum_tools.hypergraph.inequalities import *
from quantum_tools.utilities import rmt
from scipy import sparse
import numpy as np
from scipy import io
import os
from functools import reduce
from operator import mul

In [6]:
rvc = RandomVariableCollection.new(('A', 'B', 'C'), (4, 4, 4))

In [49]:
def basis_to_param(basis):
    U = np.hstack((np.kron(i, j) for i, j in basis))
    return np.append(U.real.flatten(), U.imag.flatten())

In [7]:
ei = utils.ei
pi = np.pi
perm = utils.get_triangle_permutation()

# === RHS A ===
# Eigenvectors of sigma_x
e_x_0 = (qb0 + qb1)/(sqrt2)
e_x_1 = (-qb0 + qb1)/(sqrt2)
# Eigenvectors of sigma_y
e_y_0 = (i*qb0 + qb1)/(sqrt2)
e_y_1 = (-i*qb0 + qb1)/(sqrt2)
# =============

# === LHS B ===
# Eigenvectors of -(sigma_y + sigma_x)/sqrt2
e_yx_0 = (ei(-3/4*pi)*qb0 + qb1)/(sqrt2)
e_yx_1 = (ei(1/4*pi)*qb0 + qb1)/(sqrt2)
# Eigenvectors of (sigma_y - sigma_x)/sqrt2
e_xy_0 = (ei(-1/4*pi)*qb0 + qb1)/(sqrt2)
e_xy_1 = (ei(-5/4*pi)*qb0 + qb1)/(sqrt2)
# =============

rho0 = utils.ket_to_dm(qb0)
rho1 = utils.ket_to_dm(qb1)

# phi0 = utils.ket_to_dm(e3)
# phi1 = utils.ket_to_dm(e4)
# omega0 = utils.ket_to_dm(e5)
# omega1 = utils.ket_to_dm(e6)

A_measurements = [
    utils.tensor(rho1, utils.ket_to_dm(e_y_0)),
    utils.tensor(rho1, utils.ket_to_dm(e_y_1)),
    utils.tensor(rho0, utils.ket_to_dm(e_x_1)),
    utils.tensor(rho0, utils.ket_to_dm(e_x_0)),
]
B_measurements = [
    utils.tensor(utils.ket_to_dm(e_xy_0), rho0),
    utils.tensor(utils.ket_to_dm(e_xy_1), rho0),
    utils.tensor(utils.ket_to_dm(e_yx_0), rho1),
    utils.tensor(utils.ket_to_dm(e_yx_1), rho1),
]
C_measurements = [
    utils.tensor(rho0, rho1),
    utils.tensor(rho0, rho0),
    utils.tensor(rho1, rho1),
    utils.tensor(rho1, rho0),
]

A = Measurement(A_measurements)
B = Measurement(B_measurements)
C = Measurement(C_measurements)
rhoAB = State.Strats.Deterministic.mebs(3)
rhoBC = State.Strats.Deterministic.mebs(0)
rhoAC = State.Strats.Deterministic.mebs(0)

qc = QuantumContext(random_variables=rvc, measurements=(A,B,C), states=(rhoAB, rhoBC, rhoAC), permutation=perm.T)
pd = QuantumProbDist(qc)

In [92]:
param_A = basis_to_param((
(qb1, e_y_0),
(qb1, e_y_1),
(qb0, e_x_1),
(qb0, e_x_0),
))
param_B = basis_to_param((
(e_xy_0, qb0),
(e_xy_1, qb0),
(e_yx_0, qb1),
(e_yx_1, qb1),
))
param_C = basis_to_param((
(qb0, qb1),
(qb0, qb0),
(qb1, qb1),
(qb1, qb0),
))
param_ABC = np.hstack((param_A, param_B, param_C))

In [91]:
AB_param = np.array([0,0,0,0,0,0,-1/sqrt2,0,+1/sqrt2,0,0,0,0,0,0,0])
BC_param = np.array([0,0,0,0,0,0,0,0,0,+1/sqrt2,0,0,0,0,0,+1/sqrt2])
AC_param = np.array([0,0,0,0,0,0,0,0,0,+1/sqrt2,0,0,0,0,0,+1/sqrt2])

print(np.allclose(rhoAB.data, utils.cholesky(AB_param)))
print(np.allclose(rhoBC.data, utils.cholesky(BC_param)))
print(np.allclose(rhoAC.data, utils.cholesky(AC_param)))

param_rho = np.hstack((AB_param,BC_param,AC_param))

True
True
True


In [95]:
np.hstack((param_ABC, param_rho))

array([ 0.     ,  0.     , -0.70711,  0.70711,  0.     ,  0.     ,  0.70711,  0.70711,  0.     ,  0.     , -0.     ,
        0.     ,  0.70711,  0.70711,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,
        0.     ,  0.     ,  0.70711, -0.70711,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.5    ,
       -0.5    ,  0.     ,  0.     ,  0.     , -0.     , -0.5    ,  0.5    ,  0.70711,  0.70711,  0.     ,  0.     ,
        0.     ,  0.     ,  0.70711,  0.70711, -0.5    ,  0.5    , -0.     ,  0.     ,  0.     ,  0.     , -0.5    ,
        0.5    ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  1.     ,
        0.     ,  0.     ,  1.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  1.     ,  0.     ,
        0.     ,  1.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,
        0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.    

In [93]:
print(pd)

=== ProbDist ===
RandomVariableCollection
3 Random Variables:
Outcomes: 64 = 4^3
A -> [0, 1, 2, 3]
B -> [0, 1, 2, 3]
C -> [0, 1, 2, 3]
16 Achievable outcomes.
{outcome} -> {probability}
[0, 0, 0] -> 0.1066941738241591
[0, 1, 0] -> 0.018305826175840763
[0, 2, 2] -> 0.1066941738241591
[0, 3, 2] -> 0.018305826175840756
[1, 0, 0] -> 0.018305826175840756
[1, 1, 0] -> 0.1066941738241591
[1, 2, 2] -> 0.01830582617584075
[1, 3, 2] -> 0.1066941738241591
[2, 0, 1] -> 0.1066941738241591
[2, 1, 1] -> 0.01830582617584075
[2, 2, 3] -> 0.018305826175840756
[2, 3, 3] -> 0.1066941738241591
[3, 0, 1] -> 0.01830582617584075
[3, 1, 1] -> 0.10669417382415913
[3, 2, 3] -> 0.1066941738241591
[3, 3, 3] -> 0.01830582617584075
